In [12]:
import torch
import json
import linecache
import collections

In [58]:
with open("comparisons/batch10.json", "r") as file:
    for i, json_str in enumerate(file):
        obj = json.loads(json_str)
        print(obj.keys())
        print(obj["info"])
        print(obj["summaries"])
        print(obj["choice"])
        break

dict_keys(['info', 'split', 'summaries', 'choice', 'worker', 'batch', 'extra'])
{'id': 't3_52mb8y', 'post': "This is my first post so please be kind :)\n\nI know that lots of people often feel confused when they come out of a long-term relationship. They think they have forgotten how to be single, or how to flirt/date.\n\nI am one of these people.\n\nThe problem is, my relationship started when I had just turned 16. I have never been single - as an adult. That might sound silly. But the only time I have ever flirted or dated was as an over-confident, hormone-riddled teenager.\n\nNow I have a pretty demanding job, responsibilities blah blah... And I just don't know how to this!\n\nI'm no way in a rush to get into a new relationship, but that doesn't mean I want to be completely alone in the mean time.\n\nIf anyone has experienced anything similar, or just generally has some advice, it would be greatly appreciated!", 'title': "I [23F] have just come out of 8 year relationship. Feel like 

In [55]:
class ComparisonDataset(torch.utils.data.Dataset):
    def __init__(self, path_to_dataset_dir):
        self.path_to_dataset_dir = path_to_dataset_dir
        self.file_names = [f"{path_to_dataset_dir}/batch{i}.json" for i in range (3, 11)]
        self.file_lengths = None

    def __len__(self):
        if self.file_lengths is None:
            self.file_lengths = collections.OrderedDict()
            for file_name in self.file_names:
                with open(file_name) as f:
                    self.file_lengths[file_name] = sum(1 for line in f)
        
        return sum(self.file_lengths.values())
    
    def __getitem__(self, i):
        i = i % len(self)

        if not (0 <= i < len(self)):
            raise IndexError(f"Tried to retrieve sample at index {i}, but only indicies between 0 and {len(self)-1} modulo {len(self)} are valid.")
        
        cum_length = 0
        
        for file_name in self.file_names:
            cum_length += self.file_lengths[file_name]
            if i < cum_length:
                file_idx = i - cum_length + self.file_lengths[file_name]
                # print(file_idx)
                line = linecache.getline(file_name, lineno=file_idx+1)
                # print(file_name)
                # print(line)
                payload = json.loads(line)
                choice = payload["choice"]
                summary_good = payload["summaries"][choice]["text"]
                summary_bad = payload["summaries"][1 - choice]["text"]
                post = payload["info"]["post"]
                post_good = f"{post} TLDR:{summary_good}"
                post_bad = f"{post} TLDR:{summary_bad}"
                return post_good, post_bad


dataset = ComparisonDataset(path_to_dataset_dir="./comparisons")

In [56]:
dataset[0]

("My boyfriend and I are long distance. We have a trip planned this summer which involves me going over to him in the USA. This will be the second time I have actually been with him in person. I am flying from the UK with my mum to the east coast. The original plan was for me to fly over to my boyfriend in the west coast (my parents are holidaying on the east coast) but because my mum was freaking out so much about me going to meet my boyfriend i said we can all road trip there together. I even invited her on the trip with us. I have given her all of our dates so that she can travel around with us.\n\nThe plan was for me to stay on the 4th July and fly back on the 5th. Mum knew this. I told her I had booked a flight back already from the west coast to east coast (where she would pick me up and we would fly back to the UK together). She has gone mad at me because she can't believe I would book a flight when she told me she didn't want me flying on my own. At the time I had booked it she